In [ ]:

from __future__ import print_function

import sys

# sys.path.extend(['/home/jogi/git/repository/dcnn_acdc'])
sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler, test_ensemble
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Assess performance of base and MC-dropout models on left out test set 

#### Experiments

All models were trained on fold 2 of the AD-DC data

In [32]:
# Constants aka fixed variables
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# IMPORTANT - THE FOLD WE'RE USING FOR TESTING. NOTE: Training and testing fold should be the same
# we raise an error in the test_ensemble method if this is not the case!
TEST_FOLD = 0

# baseline: 
exp_base = {2: "20180328_10_53_16_dcnnv1_150000E_lr2e02",
            0: "20180330_09_56_39_dcnnv1_150000E_lr2e02"}
exp_base_path = os.path.join(LOG_DIR, exp_base[TEST_FOLD])

# MC dropout models
# Fold 2: 
# Fold 0: 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
exp_mc005 = {2: "",
             0: "20180413_16_36_58_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_model = os.path.join(LOG_DIR, exp_mc005[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180416_08_25_43_dcnn_mcv1_150000E_lr2e02
# exp_mc01 = {2: "",
#            0: "20180416_08_25_43_dcnn_mcv1_150000E_lr2e02"}
# exp_mc01_model = os.path.join(LOG_DIR, exp_mc01[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180413_17_03_22_dcnn_mcv1_150000E_lr2e02
exp_mc005_g1000 = {2: "",
                   0: "20180413_17_03_22_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_g1000_model = os.path.join(LOG_DIR, exp_mc005_g1000[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180413_17_03_46_dcnn_mcv1_150000E_lr2e02
exp_mc005_g2000 = {2: "",
                   0: "20180413_17_03_46_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_g2000_model = os.path.join(LOG_DIR, exp_mc005_g2000[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180415_16_04_41_dcnn_mcv1_150000E_lr2e02
exp_mc005_g5000 = {2: "",
                   0: "20180415_16_04_41_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_g5000_model = os.path.join(LOG_DIR, exp_mc005_g5000[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180414_20_20_49_dcnn_mcv1_150000E_lr2e02
exp_mc005_g10000 = {2: "",
                   0: "20180414_20_20_49_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_g10000_model = os.path.join(LOG_DIR, exp_mc005_g10000[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180418_15_02_05_dcnn_mcv1_150000E_lr2e02
exp_mc01_brier = {2: "",
                    0: "20180418_15_02_05_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_brier_path = os.path.join(LOG_DIR, exp_mc01_brier[TEST_FOLD])



## Load experimental data and test data set

In [33]:
print("!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD {}".format(TEST_FOLD))

exper_base = ExperimentHandler.load_experiment(exp_base_path)
exper_hdl_base = ExperimentHandler(exper_base, use_logfile=False)
exper_hdl_base.set_root_dir(ROOT_DIR)
exper_hdl_base.set_model_name("DCNN baseline fold-{}".format(exper_hdl_base.exper.run_args.fold_ids[0]))
print("Model name {}".format(exper_hdl_base.exper.model_name))

exper_mc005 = ExperimentHandler.load_experiment(exp_mc005_model)
exper_hdl_mc005 = ExperimentHandler(exper_mc005, use_logfile=False)
exper_hdl_mc005.set_root_dir(ROOT_DIR)
exper_hdl_mc005.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc005.exper.run_args.fold_ids[0],
    exper_hdl_mc005.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005.exper.model_name))


exper_mc005_g1000 = ExperimentHandler.load_experiment(exp_mc005_g1000_model)
exper_hdl_mc005_g1000 = ExperimentHandler(exper_mc005_g1000, use_logfile=False)
exper_hdl_mc005_g1000.set_root_dir(ROOT_DIR)
exper_hdl_mc005_g1000.set_model_name("MC-dropout-g{} fold-{} (p={})".format(
                                     exper_hdl_mc005_g1000.exper.run_args.chkpnt_freq,
                                     exper_hdl_mc005_g1000.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_g1000.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_g1000.exper.model_name))

exper_mc005_g2000 = ExperimentHandler.load_experiment(exp_mc005_g2000_model)
exper_hdl_mc005_g2000 = ExperimentHandler(exper_mc005_g2000, use_logfile=False)
exper_hdl_mc005_g2000.set_root_dir(ROOT_DIR)
exper_hdl_mc005_g2000.set_model_name("MC-dropout-g{} fold-{} "
                                     "(p={})".format(exper_hdl_mc005_g2000.exper.run_args.chkpnt_freq,
                                         exper_hdl_mc005_g2000.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_g2000.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_g2000.exper.model_name))

exper_mc005_g5000 = ExperimentHandler.load_experiment(exp_mc005_g5000_model)
exper_hdl_mc005_g5000 = ExperimentHandler(exper_mc005_g5000, use_logfile=False)
exper_hdl_mc005_g5000.set_root_dir(ROOT_DIR)
exper_hdl_mc005_g5000.set_model_name("MC-dropout-g{} fold-{} "
                                     "(p={})".format(exper_hdl_mc005_g5000.exper.run_args.chkpnt_freq,
                                     exper_hdl_mc005_g5000.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_g5000.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_g5000.exper.model_name))

exper_mc005_g10000 = ExperimentHandler.load_experiment(exp_mc005_g10000_model)
exper_hdl_mc005_g10000 = ExperimentHandler(exper_mc005_g10000, use_logfile=False)
exper_hdl_mc005_g10000.set_root_dir(ROOT_DIR)
exper_hdl_mc005_g10000.set_model_name("MC-dropout-g{} fold-{} "
                                     "(p={})".format(exper_hdl_mc005_g10000.exper.run_args.chkpnt_freq,
                                         exper_hdl_mc005_g10000.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_g10000.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_g10000.exper.model_name))

exper_mc01_brier = ExperimentHandler.load_experiment(exp_mc01_brier_path)
exper_hdl_mc01_brier = ExperimentHandler(exper_mc01_brier, use_logfile=False)
exper_hdl_mc01_brier.set_root_dir(ROOT_DIR)
exper_hdl_mc01_brier.set_model_name("MC-dropout brier fold-{} (p={})".format(
                                exper_hdl_mc01_brier.exper.run_args.fold_ids[0],
                                exper_hdl_mc01_brier.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_brier.exper.model_name))

try:
    del test_set
except:
    pass

test_set = ACDC2017TestHandler(exper_config=exper_hdl_mc005_g1000.exper.config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=[TEST_FOLD],
                               debug=False, batch_size=25, use_cuda=True, load_train=False, load_val=True,
                               use_iso_path=True)

  0%|          | 0/25 [00:00<?, ?it/s]

!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD 0
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/stats/exper_stats.dll
Model name DCNN baseline fold-0
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180413_16_36_58_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-0 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180413_17_03_22_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout-g1000 fold-0 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180413_17_03_46_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout-g2000 fold-0 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180415_16_04_41_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout-g5000 fold-0 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180414_20_20_49_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.

100%|██████████| 25/25 [00:01<00:00, 15.57it/s]

INFO - Successfully loaded 25 ED/ES patient pairs


## Run model evaluation of test set

In [ ]:
mc_samples = 10
sample_weights = False
store_details = True
image_range = None
if not sample_weights:
    generate_stats = False
else:
    generate_stats = True
save_results = False
checkpoints = [150000]  #  [150000]
use_uncertainty = True

exper_handler = exper_hdl_mc01

exper_handler.logger = None   # exper_hdl_base
exper_handler.exper.outliers_per_epoch = None

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,  
              use_uncertainty=use_uncertainty, referral_threshold=0., image_range=image_range, 
              checkpoints=checkpoints,
              verbose=True, reset_results=True, store_details=store_details, 
              generate_stats=generate_stats, save_results=save_results, do_filter=True)


    
print("Ready")

In [ ]:
print(len(exper_handler.test_results.image_names))
exper_handler.test_results.compute_mean_stats()
exper_handler.test_results.show_results()

In [ ]:
exper_handler = exper_hdl_mc005

image_ids = exper_handler.test_results.image_ids

image_range = None
u_type = "stddev"

analyze_slices(exper_handler, image_range=image_range, do_save=False, do_show=True, u_type=u_type,
               translate_img_range=True)

In [ ]:
sample_weights = True
mc_samples = 50
store_details = False
image_range = None  # [5, 9, 14]
generate_stats = False

save_results = False
#  exper_hdl_mc01 exper_hdl_mc025 exp_hdl_mc01_bs  exper_hdl_mc04  exp_hdl_mc01_bs
exper_handler = exper_hdl_mc005
exper_handler.logger = None

print("INFO - Evaluating model {}".format(exper_handler.exper.model_name))

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,
              use_uncertainty=False, referral_threshold=0., image_range=image_range, verbose=False, 
              reset_results=True, use_seed=True,
              store_details=store_details, generate_stats=generate_stats, save_results=save_results,
              do_filter=True, checkpoints=[150000], u_threshold=0.)


    
print("Ready")

In [ ]:
sample_weights = True
mc_samples = 10
store_details = True
image_range = [5, 9, 14]
generate_stats = True

save_results = False
#  exper_hdl_mc01 exper_hdl_mc025 exp_hdl_mc01_bs  exper_hdl_mc04  exp_hdl_mc01_bs
exper_handler = exper_hdl_mc005
exper_handler.logger = None

print("INFO - Evaluating model {}".format(exper_handler.exper.model_name))

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,
              use_uncertainty=False, referral_threshold=0., image_range=image_range, verbose=False, 
              reset_results=True, use_seed=True,
              store_details=store_details, generate_stats=generate_stats, save_results=save_results,
              do_filter=True, checkpoints=[150000], u_threshold=0.1)


    
print("Ready")



# Compare performance of models

In [ ]:
# info_type="uncertainty"  or probabilities
# model_names a) "Baseline with p=0.5" b) MC-dropout 0.1  c) MC-dropout-Brier 0.1 
handler = exper_hdl_mc01 # exper_hdl_mc01   exper_hdl_base  exp_hdl_mc01_bs  exper_hdl_mc005

for img_idx in image_range:
    handler.test_results.visualize_uncertainty_stats(image_num=img_idx, width=16, height=10, 
                                                     info_type="uncertainty",
                                                     do_save=True, do_show=False,
                                                     model_name=handler.exper.model_name)

In [22]:
# model_name a) Baseline  b) MC-dropout 0.1
exper_handler = exper_hdl_base  #  exper_hdl_mc01  exper_hdl_base  exp_hdl_mc01_bs
model_name = exper_handler.exper.model_name
errors_only = True  # in case of the BASELINE model we only want the seg-errors
image_range = None
if image_range is None:
    image_range = np.arange(len(exper_handler.test_results.image_names))

print(image_range)
for image_num in image_range:
    exper_handler.test_results.visualize_uncertainty_histograms(image_num=image_num, width=20, height=60, 
                                                                 info_type="uncertainty",
                                                                 do_save=True, slice_range=None,
                                                                 std_threshold=0., errors_only=errors_only, 
                                                                 do_show=False, use_bald=True,
                                                                 model_name = model_name)
                                                                 # model_name="MC-dropout (0.1)") [0,1,2,3,4,5]
    print("INFO - Ready!")


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
WARNING - setting use_bald to False
Rows/columns 4/4
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient016/analysis_seg_err_slice1_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient016/analysis_seg_err_slice2_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient016/analysis_seg_err_slice3_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient016/analysis_seg_err_slice4_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient016/analysis_seg_err_slice5_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home

INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice2_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice3_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice4_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice5_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice6_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient036/analysis_seg_err_slice7_mc1_w_uncrty.pd

INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient056/analysis_seg_err_slice8_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient056/analysis_seg_err_slice9_mc1_w_uncrty.pdf
INFO - Ready!
WARNING - setting use_bald to False
Rows/columns 4/4
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient057/analysis_seg_err_slice1_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient057/analysis_seg_err_slice2_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient057/analysis_seg_err_slice3_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000

INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice3_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice4_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice5_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice6_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice7_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient077/analysis_seg_err_slice8_mc1_w_uncrty.pd

INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice1_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice2_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice3_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice4_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice5_mc1_w_uncrty.pdf
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180330_09_56_39_dcnnv1_150000E_lr2e02/figures/patient097/analysis_seg_err_slice6_mc1_w_uncrty.pd

In [ ]:
exper_hdl_mc01.test_results.visualize_test_slices(width=20, height=80, slice_range=[0, 5],
                                               do_save=False, image_num=0)

In [ ]:
exper_handler = exper_hdl_mc005_bs
exper_handler.test_results.visualize_prediction_uncertainty(width=20, height=25, slice_range=np.arange(10), 
                                                            do_save=True, std_threshold=None, 
                                                            image_num=0)

In [ ]:
Test img/slice 24/5 	ES: Dice (RV/Myo/LV)*0.84*/*0.90*/*0.95*	ED: dice (RV/Myo/LV)*0.89*/0.88/*0.97*
Test img/slice 24/6 	ES: Dice (RV/Myo/LV)*0.94*/*0.91*/*0.88*	ED: dice (RV/Myo/LV)*0.97*/0.82/*0.95*
Test img/slice 24/7 	ES: Dice (RV/Myo/LV)*0.77*/*0.68*/*0.77*	ED: dice (RV/Myo/LV)*0.93*/*0.76*/*0.90*
Test img/slice 24/8 	ES: Dice (RV/Myo/LV)*0.00*/*0.00*/*0.00*	ED: dice (RV/Myo/LV)*0.63*/*0.00*/*0.00*

In [31]:
exper_handler.test_results.test_accuracy_slices[24][:, 1:, 7]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.32285115,  0.        ,  0.        ]])